# Predicting Cattle Weight

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
import keras
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import os
from PIL import Image
import tqdm
import pandas as pd
import json
from yolo import Predictor
import time

In [ ]:
DATABASE_LOCATION = 'C:/Users/derek/Documents/cattle_data'
INPUT_PATH = DATABASE_LOCATION+'/training'
DATASET = 'training.json'

## Load Things

### Load Models

In [ ]:
model = keras.models.load_model(DATABASE_LOCATION+"/models/exceptionet_best_model")
object_detector = Predictor('C:/users/derek/Documents/nets/yolov3.weights', obj_thresh=0.01)

### Load Data

In [ ]:
f_names = []

if COMBINE_DATA_FILES:
    f_names = []
    for name in os.listdir(DATABASE_LOCATION):
        if name.endswith('.json'):
            f_names.append(name)
else:
    f_names.append(DATASET) 

frame = []
#load the dataset
with open(os.path.join(DATABASE_LOCATION, f_names.pop()), 'r') as file:
        frame = json.load(file)

# add any other datasets in the folder
for name in f_names:
    with open(os.path.join(DATABASE_LOCATION, name), 'r') as file:
        frame.extend(json.load(file))

df = pd.DataFrame(frame)
    
print('Found {} files.'.format(len(f_names)))
print('Loaded {} datapoints!'.format(len(df)))

The dataframe needs image paths so we can use the `flow_from_dataframe` function from keras. Lets create a dataframe that includes this format. Once again, we can combine the data into a single column since we will use the side and end views for the same purpose. 

For this first training process, both the side views and end views will be used in the same manner for training the dataset. We will get all the paths and preprocess the images to be the same size. 

In [11]:

side_df = df.drop(columns='end_id')
end_df = df.drop(columns='side_id')

# create training dataframe out of the two subframes
training_df = pd.concat([side_df, end_df], axis=0, ignore_index=True)

# define function for generating filename column
def create_filename(d):
    
    fnames = []
    
    for i in range(len(d)):
        
        # check if the side_id column is nan
        # for some reason the isna() function would not work here so checking the type is a work around
        if type(d.iloc[i].side_id) == float:
            
            # add path name to column where the image name includes 'end'
            fnames.append(os.path.join(INPUT_PATH, 'img', 'end_{}.png'.format(d.iloc[i].end_id)))
        
        else:
            
            # add path name to the filname column where name includes 'side'
            fnames.append(os.path.join(INPUT_PATH, 'img', 'side_{}.png'.format(d.iloc[i].side_id)))
        
    return pd.Series(fnames)


# excecute the function on the dataset
training_df['filename'] = create_filename(training_df)

# drop the extraneous columns
#train_df.drop(columns=['end_id', 'side_id'], inplace=True)

In [12]:
# look at the set where it changes from side to end
training_df[6240:6245]

,weight,side_id,end_id,filename
6240,1260.0,80715,NaN,C:/Users/derek/Documents/cattle_data/refined\i...
6241,2195.0,81352,NaN,C:/Users/derek/Documents/cattle_data/refined\i...
6242,1210.0,80308,NaN,C:/Users/derek/Documents/cattle_data/refined\i...
6243,725.0,NaN,25152,C:/Users/derek/Documents/cattle_data/refined\i...
6244,725.0,NaN,24184,C:/Users/derek/Documents/cattle_data/refined\i...


## Prediction Code

In [ ]:
# Add padding to image 
# Function taken from:
# https://note.nkmk.me/en/python-pillow-add-margin-expand-canvas/
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [ ]:
def draw_boxes(image, predictions):
    
    images = tf.image.draw_bounding_boxes(images, boxes, colors, name=None)
    
    color = (255, 0, 25)
    
    for pred in predictions:
        cv2.rect(image,  (pred['Box']['x'], pred['Box']['y']), (color))

In [ ]:
def get_regions_from_df(d):
    pred = d.prediction
    XPAD = 10
    YPAD = 20
    
    img = []
    
    for n in range(len(pred)):

        if type(pred) == pd.core.frame.DataFrame:
            pred = pred.iloc[n]

        # Load the image 
        im = cv2.imread('{}/{}/im_{}.png'.format(INPUT_PATH, d.auction, d.img_id))

        # crop the image 
        im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        im = im[max(pred['Box']['x'][1] - YPAD*2, 0): min(pred['Box']['y'][1] + YPAD, im.shape[0]), 
                                     max(pred['Box']['x'][0] - XPAD*4, 0): min(pred['Box']['y'][0] + XPAD, im.shape[1])]
        
        img.append(expand2square(im, (0,0,0)))
        
    
    return img

In [ ]:
def get_regions_from_image(image):
    
    XPAD = 10
    YPAD = 20
    
    pred = object_detector.predict_cow_info(image=image)
    
    img = []
    
    for n in range(len(pred)):

        # get a single prediction
        p = pred[n]

        # crop the image 
        im = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        im = im[max(pred['Box']['x'][1] - YPAD*2, 0): min(pred['Box']['y'][1] + YPAD, im.shape[0]), 
                                     max(pred['Box']['x'][0] - XPAD*4, 0): min(pred['Box']['y'][0] + XPAD, im.shape[1])]
        
        img.append(expand2square(im, (0,0,0)))
        
    return img

In [ ]:
def predict(model, image, jitter=False):
    
    
    

### See the results